**Vector Stores and Retrievers**

The LangChain's vector store and retrievers abstractions are designed to support retrieval of data `from vector databases and other sources` for integration with LLM workflows. \
They are important for applications that fetch data to be reasoned over as part of model inference, as in the case of Retrieval-Augmented Generation (RAG).

Topic Cover:
* Documents
* Vector Stores
* Retrievers

**Documents:** 

LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata.\
It has two attributes:
* **page_content:** a string representating the content.
* **metadata:** a dictionary containing arbitrary metadata. The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. 

Note that an individual Document object often represents a chunk of a larger document.


In [1]:
from langchain_core.documents import Document

documents=[
  Document(
    page_content="Dogs are great companions, known for their loyalty and friendliness",
    metadata={"source":"mammal-pets-doc"}
  ),
  Document(
    page_content="Cats are independent pets that often enjoy their own space",
    metadata={"source":"mammal-pets-doc"}
  ),
  Document(
    page_content="Goldfish are popular pets for beginners, requiring relatively simple care",
    metadata={"source":"fish-pets-doc"}
  ),
  Document(
    page_content="Parrots are intelligent birds capable of mimicking human speech",
    metadata={"source":"bird-pets-doc"}
  )
]

In [2]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech')]

Vector Store

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key=os.getenv("GROQ_API_KEY")

os.environ["HUGGING_FACE_ACCESS_TOKEN"]=os.getenv("HUGGING_FACE_ACCESS_TOKEN")

llm=ChatGroq(model="Llama3-8b-8192",groq_api_key=groq_api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000208E37D8250>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000208E37D9600>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [9]:
from langchain_chroma import Chroma

vector_store=Chroma.from_documents(documents,embedding=embeddings)
vector_store

In [10]:
vector_store.similarity_search("cat")

[Document(id='c498c825-df51-405c-aac9-75113d505f69', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
 Document(id='4279a498-5edd-4e08-803c-b6fe70ddb789', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness'),
 Document(id='3824102f-89ed-4cff-9fb8-6ceb254770a0', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech'),
 Document(id='37815421-7c63-4692-8dec-0a27845ee3f1', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care')]

Async Query

In [11]:
await vector_store.asimilarity_search("dog")

[Document(id='4279a498-5edd-4e08-803c-b6fe70ddb789', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness'),
 Document(id='c498c825-df51-405c-aac9-75113d505f69', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
 Document(id='37815421-7c63-4692-8dec-0a27845ee3f1', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care'),
 Document(id='3824102f-89ed-4cff-9fb8-6ceb254770a0', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech')]

In [12]:
vector_store.similarity_search_with_score("cat")

[(Document(id='c498c825-df51-405c-aac9-75113d505f69', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
  0.9436445236206055),
 (Document(id='4279a498-5edd-4e08-803c-b6fe70ddb789', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness'),
  1.5413691997528076),
 (Document(id='3824102f-89ed-4cff-9fb8-6ceb254770a0', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech'),
  1.6284855604171753),
 (Document(id='37815421-7c63-4692-8dec-0a27845ee3f1', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care'),
  1.7496694326400757)]

---
**Retrievers**

LangChain VectorStore objects do not subclass Runnable, and so cannot immediately be integrated into LangChain Expression Language chains.

LangChain Retrievers are Runnables, so they implement a standard set of methods (e.g., synchronous and asynchronous invoke and batch operations) and are designed to be incorporated in LCEL chains.

We can create a simple version of this, without subclassing Retriever. If we choose what method we wish to use to retrieve documents, we can create a runnable easily.

In [15]:
# Building one around the similarity_search method
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

# Naive Approach
retriever=RunnableLambda(vector_store.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])


[[Document(id='c498c825-df51-405c-aac9-75113d505f69', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space')],
 [Document(id='4279a498-5edd-4e08-803c-b6fe70ddb789', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness')]]

**2nd Approach**

Vector_store implement an as_retriever method that will generate a Retriever, specifically a VectorStoreRetriever. 

These retrievers include specific search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them.\
For instance, we can replicate the above with the following:

In [16]:
retriever=vector_store.as_retriever(
  search_type="similarity",
  search_kwargs={"k":1}
)
retriever.batch(["cat","dog"])

[[Document(id='c498c825-df51-405c-aac9-75113d505f69', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space')],
 [Document(id='4279a498-5edd-4e08-803c-b6fe70ddb789', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness')]]

Integrating Retriever with Chain

**((RAG))**

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message="""
Answer this question using the provided context only.
{question}
Context:
{context}
"""

prompt=ChatPromptTemplate.from_messages(
  [("human",message)]
)

rag_chain={"context":retriever,"question":RunnablePassthrough()}|prompt|llm

response=rag_chain.invoke("tell me about dogs")

In [18]:
response.content

'According to the context, dogs are great companions, known for their loyalty and friendliness.'